# --- `Day 23`: Amphipod ---

In [2]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache
from ipycanvas import RoughCanvas, hold_canvas, Canvas, MultiRoughCanvas
from time import sleep

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

### Helpers

In [20]:
def createMultiCanvas(w, h):
    canvas = MultiRoughCanvas(2, width = w, height = h)
    canvas[0].font = '14px serif'
    canvas[1].font = '14px serif'
    canvas[0].fill_style = "#FFF0C9"
    canvas[0].stroke_style = "white"
    canvas[0].fill_rect(0, 0, canvas.size[0], canvas.size[1])
    return canvas

def drawBackground(canvas):
    background = [
        [True, True, True, True, True, True, True, True, True, True, True, True, True],
        [True, False, False, False, False, False, False, False, False, False, False, False, True], 
        [True, True, True, False, True, False, True, False, True, False, True, True, True],
        [False, False, True, False, True, False, True, False, True, False, True, False, False],
        [False, False, True, False, True, False, True, False, True, False, True, False, False],
        [False, False, True, False, True, False, True, False, True, False, True, False, False],
        [False, False, True, True, True, True, True, True, True, True, True, False, False],
    ]

    pixelSize = 20
    xoffset, yoffset = 90, 10
    color = "#5770B3"

    canvas.clear()
    canvas.fill_style = "#FFF0C9"
    canvas.stroke_style = "white"
    canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

    canvas.fill_style = color
    canvas.stroke_style = color

    xs = [i * pixelSize + xoffset for r in background for i,c in enumerate(r) if c]
    ys = [i * pixelSize + yoffset for i,r in enumerate(background) for c in r if c]
    canvas.fill_rects(xs, ys, pixelSize)
    canvas.stroke_rects(xs, ys, pixelSize)

def drawBoard(board, canvas, label):
    colors = {'A':"red", 'B':"green", 'C':"blue", 'D':"black", '.':"white"}

    pixelSize = 20
    xoffset, yoffset = 90, 10

    canvas.clear()
    canvas.fill_style = "#FFF0C9"
    canvas.stroke_style = "white"
    canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

    for n in range(27):
        c = board[n]
        if n > 10:
            x, y = ((n - 11) // 4) * 2 + 3, (n - 11) % 4 + 2
        else:
            x, y = n + 1, 1
        canvas.fill_style = colors[c]
        canvas.stroke_style = "black"
        if c != '.':
            canvas.stroke_rect(x * pixelSize + xoffset, y * pixelSize + yoffset, pixelSize)

        canvas.fill_text(c, x * pixelSize + xoffset + 5, ((y + 1) * pixelSize) + yoffset - 5, pixelSize)

    canvas.fill_style = "black"
    canvas.fill_text(label, 10, 20)

## Solution 1

In [21]:
def updateBoard(board, position):
    source, dest = position

    newBoard = [x for x in board]
    piece = newBoard[source]
    newBoard[dest] = piece
    newBoard[source] = '.'
    
    return tuple(newBoard)

def getNeighbors(board, position):
    caves = {2:11, 4:15, 6:19, 8:23, 11:2, 15:4, 19:6, 23:8}
    neighbors = []
    if position < 11:
        if position < 10:
            neighbors.append(position + 1)
        if position > 0:
            neighbors.append(position - 1)
        if position in caves:
            neighbors.append(caves[position])
    else:
        if position in [11,15,19,23]:
            neighbors.append(caves[position])
            neighbors.append(position + 1)
        elif position in [14,18,22,26]:
            neighbors.append(position - 1)
        else:
            neighbors.append(position + 1)
            neighbors.append(position - 1)
    return [n for n in neighbors if board[n] == '.']

def validPath(board, piece, path):
    caves = {'A':[11,12,13,14], 'B':[15,16,17,18], 'C':[19,20,21,22], 'D':[23,24,25,26]}
    _, dest = path[-1]
    source = path[0][0]
    if dest in [2, 4, 6, 8]:
        return False
    if dest > 10 and dest not in caves[piece]:
        return False
    if dest in [11, 15, 19, 23] and (board[dest + 1] != piece or board[dest + 2] != piece or board[dest + 3] != piece):
        return False
    if dest in [12, 16, 20, 24] and (board[dest + 1] != piece or board[dest + 2] != piece):
        return False
    if dest in [13, 17, 21, 25] and board[dest + 1] != piece:
        return False
    if source < 11 and dest < 11:
        return False
    if source in caves[piece] and dest in caves[piece]:
        return False
    return True

def getPaths(board, start):
    paths = []
    q = deque()
    q.append((start, []))
    visited = set()
    piece = board[start]
    while q:
        position, path = q.popleft()
        neighbors = getNeighbors(board, position)
        for n in neighbors:
            if not n in visited:
                visited.add(n)
                newPath = path + [(position, n)]
                if validPath(board, piece, newPath):
                    paths.append(newPath)
                q.append((n, newPath))

    directToFinal = [p for p in paths if p[-1][1] > 10]
    if len(directToFinal) > 0:
        return directToFinal

    return paths

def canMove(board, position):
    caves = {'A':[11,12,13,14], 'B':[15,16,17,18], 'C':[19,20,21,22], 'D':[23,24,25,26]}
    piece = board[position]
    if piece == '.':
        return False
    cave = caves[piece]
    if position in cave:
        if board[cave[0]] == board[cave[1]] == board[cave[2]] == board[cave[3]] == piece:
            return False
        if position == cave[1] and board[cave[1]] == board[cave[2]] == board[cave[3]] == piece:
            return False
        if position == cave[2] and board[cave[2]] == board[cave[3]] == piece:
            return False
        if position == cave[3] and board[cave[3]] == piece:
            return False
    neighbors = getNeighbors(board, position)
    if len(neighbors) == 0:
        return False
    return True

def gameOver(board):
    caves = {'A':[11,12,13,14], 'B':[15,16,17,18], 'C':[19,20,21,22], 'D':[23,24,25,26]}
    for piece, positions in caves.items():
        if not all(board[x] == piece for x in positions):
            return False
    return True

def play(board, DP, level = 1):
    if board in DP:
        return DP[board]
    costs = {'A':1, 'B':10, 'C':100, 'D':1000}

    if gameOver(board):
        DP[board] = [[], 0]
        return DP[board]

    moves = []
    for i in range(len(board)):
        if canMove(board, i):
            paths = getPaths(board, i)
            moves.extend(paths)

    if len(moves) == 0:
        DP[board] = None
        return DP[board]

    best = None
    for move in moves:
        newBoard = board
        position = (move[0][0], move[-1][1])
        piece = newBoard[move[0][0]]
        newBoard = updateBoard(newBoard, position)

        temp = play(newBoard, DP, level + 1)
        if temp != None:
            path, childCost = temp
            cost = costs[piece] * len(move) + childCost
            if best == None or cost < best[1]:
                path = path[:]
                path.append(move)
                best = (path, cost)

    DP[board] = best
    return DP[board]

def solve(caves):
    hallway = ['.'] * 11
    board = tuple(hallway + caves)

    pixelSize = 20
    w,h = 13, 7
    canvas = createMultiCanvas(w * pixelSize + 100, h * pixelSize + 20)
    display(canvas)
    drawBackground(canvas[0])
    drawBoard(board, canvas[1], "0: 0")

    DP = {}
    paths, totalCost = play(board, DP)

    i = 0
    drawBoard(board, canvas[1], i)
    sleep(1.0)

    costs = {'A':1, 'B':10, 'C':100, 'D':1000}
    cost = 0
    newBoard = board
    for path in reversed(paths):
        i += 1
        for position in path:
            with hold_canvas(canvas):
                sleep(0.08)
                piece = newBoard[position[0]]
                cost += costs[piece]
                newBoard = updateBoard(newBoard, position)
                drawBoard(newBoard, canvas[1], f'{i}: {cost}')
        sleep(0.4)
    return totalCost
    
# example data
solve(["B", "D", "D", "A", "C", "C", "B", "D", "B", "B", "A", "C", "D", "A", "C", "A"])

MultiRoughCanvas(height=160, width=360)

44169

In [22]:
# final data
solve(["C", "D", "D", "D", "A", "C", "B", "C", "B", "B", "A", "A", "D", "A", "C", "B"])

MultiRoughCanvas(height=160, width=360)

51436